In [ ]:
import googleapiclient.discovery
import pandas as pd 
from api_keys import YOUTUBE_API_KEY

api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY
)


In [ ]:
def get_playlist_ids(youtube, channel_names):
    playlist_ids = {}
    for channel_name in channel_names:
        request = youtube.channels().list(
            part="contentDetails",
            forUsername=channel_name
        )
        response = request.execute()
        if response['items']:
            playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
            playlist_ids[channel_name] = playlist_id #looping through each channel and sotring the channel name with playlust id as pair ie: "ufc:"Zshsxn2351vsa""
        else:
            print(f"Channel '{channel_name}' not found")
    return playlist_ids

In [ ]:
all_video_ids = {}
def get_video_ids(youtube, playlist_ids):
    for channel_name, playlist_id in playlist_ids.items(): #looping through the channel name and playlist id to gtet the values of both
        video_ids = []
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50
        )
        response = request.execute()
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        all_video_ids[channel_name] = video_ids #channel name will then get stored alongside the video id as pair 
    # print(all_video_ids)
    return all_video_ids

In [ ]:
def get_video_stats(youtube, all_video_ids):
    video_info  = []

    for channel_name, video_ids in all_video_ids.items(): # Loop through each channel and its video IDs
        for video_id in video_ids[:5]: # Loop through the first 5 video IDs
            request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=video_id
            )
            response = request.execute()
            if response['items']:
            
                video_data = response['items'][0]
                title = video_data['snippet'].get('title', 'Unknown')
                views = video_data['statistics'].get('viewCount', 0)
                likes = video_data['statistics'].get('likeCount', 0)
                dislikes = video_data['statistics'].get('dislikeCount', 0)
                comments = video_data['statistics'].get('commentCount', 0)
                tags = video_data['snippet'].get('tags', [])
                published = video_data['snippet'].get('publishedAt', 'Unknown')
                duration = video_data['contentDetails'].get('duration', 'Unknown')
                video_info.append({
                    "Channel": channel_name,
                    "Title": title,
                    "Views": views,
                    "Likes": likes,
                    "Dislikes": dislikes,
                    "comments": comments,
                    "tags":tags,
                    "duration":duration,
                    "publish date": published
                })
            else:
                print(f"Video with ID {video_id} not found")
    
    # print(video_info)
   
    return pd.DataFrame(video_info)

In [6]:

channel_names_input = input("Enter the YouTube channel names separated by commas: ")
channel_names = [name.strip() for name in channel_names_input.split(",")]

playlist_ids = get_playlist_ids(youtube, channel_names)

all_video_ids = get_video_ids(youtube, playlist_ids)

video_df = get_video_stats(youtube, all_video_ids)
video_df
video_df.isnull().any()
video_df.dtypes
numeric_cols  = ['viewCount', 'likeCount', 'favouriteCount', 'commentCount']
video_df


Enter the YouTube channel names separated by commas:  ufc,wwe


,Channel,Title,Views,Likes,Dislikes,comments,tags,duration,publish date
0,ufc,Ilia Topuria gets a taste of NFL radio row #uf...,32404,1433,0,82,"[UFC, MMA, UFC 298, Ilia Topuria, Super Bowl]",PT59S,2024-02-11T19:30:01Z
1,ufc,UFC 298 Countdown - Dvalishvili vs Cejudo | Fe...,38899,829,0,223,"[ufc, mma, ufc 298, ufc countdown, Robert Whit...",PT10M2S,2024-02-11T18:00:15Z
2,ufc,UFC 298 Countdown - Whittaker vs Costa | Co-Ma...,41974,883,0,205,"[ufc, mma, ufc 298, ufc countdown, Robert Whit...",PT10M3S,2024-02-11T16:00:24Z
3,ufc,UFC 298 Countdown - Volkanovski vs Topuria | M...,104915,2445,0,496,"[ufc, mma, ufc 298, ufc countdown, alexander v...",PT27M39S,2024-02-11T14:00:04Z
4,ufc,UFC 298 Countdown - Full Episode Premiere,427127,8734,0,1224,"[ufc, mma, ufc 298, ufc countdown, alexander v...",PT47M42S,2024-02-11T06:00:09Z
5,wwe,Christian encourages the WWE Universe to Smack...,35153,307,0,37,"[WWE, WWFE, World Wrestling Entertainment, Wor...",PT25S,2010-04-23T06:54:29Z
6,wwe,Kofi Kingston encourages the WWE Universe to S...,43262,377,0,72,"[WWE, WWFE, World Wrestling Entertainment, Wor...",PT25S,2010-04-23T06:43:41Z
7,wwe,John Cena encourages the WWE Universe to Smack...,53448,639,0,31,"[WWE, WWFE, World Wrestling Entertainment, Wor...",PT24S,2010-04-23T06:39:55Z
8,wwe,A review of NXT Rookie Heath Slater,17832,147,0,0,"[WWE, WWFE, World Wrestling Entertainment, Wor...",PT1M25S,2010-04-22T08:20:33Z
9,wwe,Matt Hardy vs. Intercontinental Champion Drew ...,19208,199,0,0,"[WWE, WWFE, World Wrestling Entertainment, Wor...",PT2M15S,2010-04-21T08:28:55Z
